In [3]:
import org.apache.spark.ml.feature.PCA
import org.apache.spark.mllib.linalg.Vectors
import sqlContext.implicits._
val input = sc.textFile("wasb:///iris-multiclass.csv")
val vec = input.map(line => line.split(",")).map(mapped => Vectors.dense(Array(mapped(0).toDouble, mapped(1).toDouble, mapped(2).toDouble, mapped(3).toDouble)))


vec: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[7] at map at <console>:32

In [4]:
vec.take(5)

res1: Array[org.apache.spark.mllib.linalg.Vector] = Array([5.1,3.5,1.4,0.2], [7.0,3.2,4.7,1.4], [6.3,3.3,6.0,2.5], [4.9,3.0,1.4,0.2], [6.4,3.2,4.5,1.5])

In [5]:
import sqlContext.implicits._
val data = vec.collect()
val df = sqlContext.createDataFrame(data.map(Tuple1.apply)).toDF("features")


df: org.apache.spark.sql.DataFrame = [features: vector]

In [6]:
val pca = new PCA().setInputCol("features").setOutputCol("outfeatures").setK(2)
val pcsModel = pca.fit(df)


pcsModel: org.apache.spark.ml.feature.PCAModel = pca_782ddc0e26b1

In [7]:
val pcaDF = pcsModel.transform(df)
val result = pcaDF.select("outfeatures")
result.take(5)


res2: Array[org.apache.spark.sql.Row] = Array([[-2.827135972679015,-5.641331045573371]], [[-6.796137685628032,-6.0001629169419175]], [[-8.043070078222522,-5.3028814941686795]], [[-2.795952482148833,-5.145166883252961]], [[-6.44375385076588,-5.633921820642639]])

In [8]:
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val matrix = new RowMatrix(vec)
val comps = matrix.computePrincipalComponents(2)


comps: org.apache.spark.mllib.linalg.Matrix = 
-0.36158967738144987  -0.6565398832858255  
0.0822688898922141    -0.7297123713265027  
-0.8565721052905283   0.17576740342864972  
-0.3588439262482152   0.07470647013503745